In [2]:
weather = pd.read_csv('open-meteo-all-data.csv', parse_dates=['time'], index_col='time')

In [3]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file into a pandas DataFrame
# 'timestamp' column is parsed as dates and set as the index
df = pd.read_csv('cop_january.csv', parse_dates=['timestamp'], index_col='timestamp')

# Ensure the index is in datetime format
if not isinstance(df.index, pd.DatetimeIndex):
    df.index = pd.to_datetime(df.index)

# Calculate COP (Coefficient of Performance) for each group
# COP is the ratio of heat output to electricity input
df['top20cop'] = df['top20heat'] / df['top20elec']
df['top50cop'] = df['top50heat'] / df['top50elec']
df['allcop'] = df['allheat'] / df['allelec']

# Calculate COP for the differences between all data and top 20%/50%
df['all_minus_top20_elec'] = df['allelec'] - df['top20elec']
df['all_minus_top20_heat'] = df['allheat'] - df['top20heat']
df['all_minus_top20_cop'] = df['all_minus_top20_heat'] / df['all_minus_top20_elec']

df['all_minus_top50_elec'] = df['allelec'] - df['top50elec']
df['all_minus_top50_heat'] = df['allheat'] - df['top50heat']
df['all_minus_top50_cop'] = df['all_minus_top50_heat'] / df['all_minus_top50_elec']

# Ensure index is in datetime format (redundant if done earlier)
df.index = pd.to_datetime(df.index)
weather.index = pd.to_datetime(weather.index)

# Handle timezone information to ensure consistency between datasets
if df.index.tz is not None and weather.index.tz is None:
    weather.index = weather.index.tz_localize(df.index.tz)
elif df.index.tz is None and weather.index.tz is not None:
    df.index = df.index.tz_localize(weather.index.tz)
elif df.index.tz != weather.index.tz:
    weather.index = weather.index.tz_convert(df.index.tz)

# Get the date range from the energy consumption dataframe
start_date = df.index.min()
end_date = df.index.max()

# Clip the weather dataframe to match the energy consumption date range
weather = weather.loc[start_date:end_date]

# Merge the energy consumption and weather dataframes
merged_df = pd.merge(df, weather, left_index=True, right_index=True, how='left')

# Drop the last row in the merged dataframe
merged_df = merged_df.drop(merged_df.index[-1])

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming df_merged is your DataFrame and its index is already a datetime

# Create the plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for apparent temperature
fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df["apparent_temperature (°C)"], name="Apparent Temperature"),
    secondary_y=False,
)

# Add trace for temperature_2m
fig.add_trace(
    go.Scatter(x=merged_df.index, y=merged_df["temperature_2m (°C)"], name="Temperature 2m"),
    secondary_y=False,
)

# Set x-axis title
fig.update_xaxes(title_text="Time")

# Set y-axes titles
fig.update_yaxes(title_text="Apparent Temperature (°C)", secondary_y=False)
fig.update_yaxes(title_text="Temperature 2m (°C)", secondary_y=True)

# Set title
fig.update_layout(title_text="Temperature Comparison Over Time")

# Show the plot
fig.show()

In [47]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import optimize
from sklearn.linear_model import LinearRegression

# Define Richards function for curve fitting
def richards(x, A, K, B, M, v):
    return A + (K - A) / (1 + v * np.exp(-B * (x - M))) ** (1 / v)

# Function to calculate R-squared
def calculate_r_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

# Function to fit Richards curve to data
def fit_richards(x, y):
    # Initialize parameters
    A_init = min(y)
    K_init = max(y)
    B_init = 0.1
    M_init = np.median(x)
    v_init = 5.0
    p0 = [A_init, K_init, B_init, M_init, v_init]
   

    # Set bounds for parameters
    bounds = ([min(y), max(y), 0, min(x), 0.1],
              [max(y), max(y)*1.5, 10, max(x), 10])
    
    
   
    try:
        # Attempt to fit the curve
        popt, _ = optimize.curve_fit(richards, x, y, p0, bounds=bounds, method='trf', maxfev=5000)
        # Calculate R-squared
        y_pred = richards(x, *popt)
        r_squared = calculate_r_squared(y, y_pred)
        return popt, r_squared
    except RuntimeError:
        print("Could not fit Richards function. Using initial guess.")
        return p0, None

# Function to fit linear regression
def fit_linear(x, y):
    model = LinearRegression()
    x_array = np.array(x).reshape(-1, 1)
    y_array = np.array(y)
    model.fit(x_array, y_array)
    y_pred = model.predict(x_array)
    r_squared = calculate_r_squared(y_array, y_pred)
    return model.coef_[0], model.intercept_, r_squared

# List of COP columns to plot
cop_columns = [
    'top20cop', 'top50cop', 'allcop',
    'all_minus_top20_cop', 'all_minus_top50_cop'
]

# Create the main figure
fig = go.Figure()

# Color palette for scatter plots and fit lines
colors = ['blue', 'red', 'green', 'purple', 'orange']

# Dictionary to store Richards parameters and R-squared values
richards_params = {}
r_squared_values = {}

# Dictionary to store linear fit parameters and R-squared values
linear_params = {}
linear_r_squared_values = {}

# Create scatter plots with Richards and linear fit lines
for i, col in enumerate(cop_columns):
    x = merged_df['apparent_temperature (°C)']
    y = merged_df[col]
   
    # Add scatter plot
    fig.add_trace(
        go.Scatter(
            x=x, y=y,
            mode='markers',
            name=f'{col} Data',
            marker=dict(color=colors[i], size=5, opacity=0.6),
        )
    )
   
    # Prepare x values for fit lines
    x_fit = np.linspace(-20, 30, 500)
   
    # Fit Richards curve and add to plot
    try:
        popt_richards, r_squared = fit_richards(x, y)
        y_fit_richards = richards(x_fit, *popt_richards)
        fig.add_trace(
            go.Scatter(x=x_fit, y=y_fit_richards, mode='lines', name=f'{col} Richards Fit', line=dict(color=colors[i], dash='solid'))
        )
       
        # Store parameters and R-squared
        key = col.replace('cop', '').strip('_')
        richards_params[key] = tuple(popt_richards)
        r_squared_values[key] = r_squared
       
    except Exception as e:
        print(f"Could not fit Richards for {col}: {str(e)}")
    
    # Fit linear regression and add to plot
    slope, intercept, linear_r_squared = fit_linear(x, y)
    y_fit_linear = slope * x_fit + intercept
    fig.add_trace(
        go.Scatter(x=x_fit, y=y_fit_linear, mode='lines', name=f'{col} Linear Fit', line=dict(color=colors[i], dash='dot'))
    )
    
    # Store linear fit parameters and R-squared
    linear_params[key] = (slope, intercept)
    linear_r_squared_values[key] = linear_r_squared

# Update layout
fig.update_layout(
    height=600,
    width=1000,
    title_text="COP vs apparent_temperature (°C) (with Richards and Linear Fit Lines)",
    xaxis_title="apparent_temperature (°C)",
    yaxis_title="COP",
    
    yaxis_range=[0, 7],
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    ),
    margin=dict(t=50)
)

# Show the plot
fig.show()

# Calculate and print average COP for each category
print("\nAverage COP for each category:")
for col in cop_columns:
    avg_cop = merged_df[col].mean()
    print(f"{col}: {avg_cop:.4f}")

# Add the additional entry for 'all_minus_top50_minus_1'
richards_params['all_minus_top50_minus_1'] = richards_params['all_minus_top50']
r_squared_values['all_minus_top50_minus_1'] = r_squared_values['all_minus_top50']
linear_params['all_minus_top50_minus_1'] = linear_params['all_minus_top50']
linear_r_squared_values['all_minus_top50_minus_1'] = linear_r_squared_values['all_minus_top50']

# Output Richards function parameters in the specified format
print("\nRichards function parameters:")
print("richards_params = {")
for key, params in richards_params.items():
    print(f"    '{key}': {params},")
print("}")

# Print R-squared values for Richards fits
print("\nR-squared values for Richards fits:")
for key, r_squared in r_squared_values.items():
    if r_squared is not None:
        print(f"{key}: {r_squared:.4f}")
    else:
        print(f"{key}: Could not calculate R-squared")

# Output Linear function parameters
print("\nLinear function parameters (slope, intercept):")
print("linear_params = {")
for key, params in linear_params.items():
    print(f"    '{key}': {params},")
print("}")

# Print R-squared values for Linear fits
print("\nR-squared values for Linear fits:")
for key, r_squared in linear_r_squared_values.items():
    print(f"{key}: {r_squared:.4f}")

# Print some basic statistics about the temperature data
print("\napparent_temperature (°C) statistics:")
print(merged_df['apparent_temperature (°C)'].describe())


Average COP for each category:
top20cop: 4.3523
top50cop: 4.0429
allcop: 3.5893
all_minus_top20_cop: 3.4805
all_minus_top50_cop: 3.3055

Richards function parameters:
richards_params = {
    'top20': (2.899540960453274, 5.871790540553472, 0.12296227280025464, -2.785892381253248, 0.10000000023407231),
    'top50': (2.7417000494678523, 5.507528926934868, 0.17833045594675145, -0.3606817676332919, 0.6153499900509748),
    'all': (2.5070737679242527, 4.8567255768288495, 0.16675698540774989, -0.9287423443134627, 0.3341547873967765),
    'all_minus_top20': (2.4123013356590453, 4.684147041930229, 0.18206035531590722, -0.5489295446817481, 0.5374930940248633),
    'all_minus_top50': (2.2903480856407357, 4.464281297664511, 0.14915045918019204, -1.6952804061999143, 0.15942252915903377),
    'all_minus_top50_minus_1': (2.2903480856407357, 4.464281297664511, 0.14915045918019204, -1.6952804061999143, 0.15942252915903377),
}

R-squared values for Richards fits:
top20: 0.6408
top50: 0.7709
all: 0.8000

In [70]:
final_df = pd.read_csv('heat_normalised_electricity_load_heat_pump.csv', parse_dates=['time'], index_col='time')    

In [24]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider, VBox, HTML
import numpy as np

# Assuming final_df is already loaded and converted to integers (watts)
# Define the columns to work with
columns = ['top20_electricity_input', 'top50_electricity_input', 'all_electricity_input', 
           'all_minus_top20_electricity_input', 'all_minus_top50_electricity_input', 
           'all_minus_top50_minus_1_electricity_input']

# Pre-compute the column names without '_electricity_input'
column_names = [col.replace('_electricity_input', '') for col in columns]

# Pre-compute the base values as a numpy array
base_values = final_df[columns].to_numpy()

# Create a FigureWidget instead of a Figure
fig = go.FigureWidget()

# Add traces to the figure
for i, column in enumerate(columns):
    fig.add_scatter(x=final_df.index, y=base_values[:, i], name=column_names[i], mode='lines')

# Initial plot setup
fig.update_layout(
    title='Grid Load Simulation for 1 Heat Pump',
    xaxis_title='Time',
    yaxis_title='Load (GW)',
    legend_title='Categories'
)

# Function to calculate statistics and update table
def calculate_statistics(num_heatpumps=1):
    # Scale base values by the number of heat pumps
    updated_values = base_values * num_heatpumps
   
    # Convert updated values back to DataFrame for easier manipulation
    updated_df = pd.DataFrame(updated_values, columns=columns, index=final_df.index)
   
    # Calculate yearly statistics
    yearly_stats = updated_df.resample('Y').agg(['max', 'mean', 'sum'])
   
    # Calculate cumulative sum for each year and convert from Wh to GWh
    cumsum_gwh = updated_df.resample('Y').apply(lambda x: x.cumsum().iloc[-1])
   
    # Add cumulative sum results to the yearly_stats DataFrame
    for col in columns:
        yearly_stats[(col, 'cumsum')] = cumsum_gwh[col]

    # Prepare HTML table with regrouped columns
    table_html = "<table><tr><th>Year</th>"
    stats = ['Max (GW)', 'Mean (GW)', 'Cum. (GWh)']
    for stat in stats:
        for col in column_names:
            table_html += f"<th>{col} {stat}</th>"
    table_html += "</tr>"
   
    for year, row in yearly_stats.iterrows():
        table_html += f"<tr><td>{year.year}</td>"
        for stat, agg_func in [('Max (GW)', 'max'), ('Mean (GW)', 'mean'), ('Cum. (GWh)', 'cumsum')]:
            for col in column_names:
                full_col = f"{col}_electricity_input"
                value = row[(full_col, agg_func)]
                table_html += f"<td>{value:.2f}</td>"
        table_html += "</tr>"
   
    table_html += "</table>"
    return table_html

# HTML widget for displaying the table
table_widget = HTML()

# Function to update both the plot and the statistics table
def update_visualization(num_heatpumps=1):
    # Update plot
    updated_values = base_values * num_heatpumps
    for i, trace in enumerate(fig.data):
        trace.y = updated_values[:, i]
    
    fig.update_layout(
        title=f'Grid Load Simulation for {num_heatpumps} Heat Pumps'
    )
    
    # Update table
    table_widget.value = calculate_statistics(num_heatpumps)

# Create slider with continuous_update set to False
slider = IntSlider(value=1, min=0, max=15, step=1, description='Number of Heat Pumps, millions', continuous_update=False)

# Create the interactive widget and link it to the update function
interact(update_visualization, num_heatpumps=slider)

# Display the plot and table
VBox([fig, table_widget])

interactive(children=(IntSlider(value=1, continuous_update=False, description='Number of Heat Pumps, millions'…

    'data': [{'mode': 'lines',
              'name': 'top20',
              'typ…